In [178]:
import pandas as pd
import numpy as np

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [180]:
df = pd.read_csv('ocean-acidification-munida-state-1998-2020.csv')
df.head()

,date,year,site,region,measure,data_value,unit,statistic,dataset
0,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,Salinity,3.426800e+01,NaN,Mean,Munida
1,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,Total alkalinity,2.288700e+03,umol/kg-1,Value,Munida
2,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,Dissolved inorganic carbon,2.084500e+03,umol/kg-1,Value,Munida
3,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,pH,8.094800e+00,NaN,Value,Munida
4,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,Hydrogen ion conc,8.039500e-09,mol/kg-1,Value,Munida


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1088 non-null   object 
 1   year        1088 non-null   int64  
 2   site        1088 non-null   object 
 3   region      1088 non-null   object 
 4   measure     1088 non-null   object 
 5   data_value  1037 non-null   float64
 6   unit        680 non-null    object 
 7   statistic   1088 non-null   object 
 8   dataset     1088 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 76.6+ KB


In [182]:
df.nunique()

date          136
year           24
site            1
region          1
measure         8
data_value    849
unit            4
statistic       2
dataset         1
dtype: int64

In [183]:
df.isna().sum()

date            0
year            0
site            0
region          0
measure         0
data_value     51
unit          408
statistic       0
dataset         0
dtype: int64

In [184]:
len(df)

1088

In [185]:
1088/8

136.0

In [186]:
df_ph = df[df['measure'] == 'pH'].copy()
df_ph.head()

,date,year,site,region,measure,data_value,unit,statistic,dataset
3,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,pH,8.0948,NaN,Value,Munida
11,1998-02-16,1998,Taiaroa Head (50km offshore),Otago,pH,8.0906,NaN,Value,Munida
19,1998-03-17,1998,Taiaroa Head (50km offshore),Otago,pH,8.1026,NaN,Value,Munida
27,1998-04-15,1998,Taiaroa Head (50km offshore),Otago,pH,NaN,NaN,Value,Munida
35,1998-10-05,1998,Taiaroa Head (50km offshore),Otago,pH,8.0743,NaN,Value,Munida


In [187]:
len(df_ph)

136

In [188]:
df_ph.isna().sum()

date            0
year            0
site            0
region          0
measure         0
data_value      9
unit          136
statistic       0
dataset         0
dtype: int64

### Data Imputation

In [189]:
from sklearn.impute import KNNImputer

1. Mean Imputation (fill in missing values with the mean of the column)

In [190]:
def impute_mean(data, column):
    data[column] = data[column].fillna(data[column].mean())
    return data

2. Median Imputation (fill in missing values with the median of the column)

In [191]:
def impute_median(data, column):
    data[column] = data[column].fillna(data[column].median())
    return data

3. Forward-fill (fill in missing values py propogating the last valid observation forward)

In [192]:
def impute_forward_fill(data, column):
    data[column] = data[column].fillna(method='ffill')
    return data

4. backward-fill (fill in missing values by propogating the next valid observation backwards)

In [193]:
def impute_backward_fill(data, column):
    data[column] = data[column].fillna(method='bfill')
    return data

5. Interpolation (fill in missing values using linear interpolation)

In [194]:
def impute_interpolation(data, column):
    data[column] = data[column].interpolate(method='linear')
    return data

6. Custom average (fill in missing values using the average of the value before and value after)
- edge cases with consecutive missing values are handled using the average of the two preceding values or using forward-fill as a fallback

In [195]:
def impute_surrounding_average(data, column):
    # Reset index temporarily to avoid index mismatch issues
    data = data.reset_index(drop=True)
    
    def surrounding_avg(series):
        for idx in series[series.isnull()].index:
            # If we can calculate the average of the values before and after
            if idx > 0 and idx < len(series) - 1 and not np.isnan(series[idx - 1]) and not np.isnan(series[idx + 1]):
                series[idx] = (series[idx - 1] + series[idx + 1]) / 2
            # Fallback: Use the average of two preceding values if possible
            elif idx > 1 and not np.isnan(series[idx - 1]) and not np.isnan(series[idx - 2]):
                series[idx] = (series[idx - 1] + series[idx - 2]) / 2
            # Final fallback: Forward-fill (use the last valid value)
            elif idx > 0:
                series[idx] = series[idx - 1]
        return series

    # Apply the surrounding average logic
    data[column] = surrounding_avg(data[column])
    data[column] = data[column].fillna(method='ffill')  # Ensure no NaNs remain with forward-fill
    return data


7. KNN Imputation (fill in missing values using K-Neareest Neighbors imputation)

In [196]:
def impute_knn(data, column, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    data[[column]] = imputer.fit_transform(data[[column]])
    return data

### Best Methods to fill in missing data in this case from ChatGPT:
1. Custom Average (surrounding average): Maintains local trends, avoids unrealistic values
2. Linear Interpolation: provides smooth estimate, preserves trend without overcomplicating imputation
3. Forward Fill: simple and aligns well with random forest, doesn't introduce artificial variability
4. KNN Imputation: uses patterns from data to estimate missing values, exploits relationships between features

# The model

In [197]:
def random_forest_time_series(data, target_col, n_lags=3, test_size=0.2, n_estimators=100):
    """
    Train a Random Forest model for time-series prediction with customizable parameters.
    
    Parameters:
        data (pd.DataFrame): The dataset containing the time-series data.
        target_col (str): The column name of the target variable.
        n_lags (int): Number of lagged features to create.
        test_size (float): Proportion of the data to use for testing (0 to 1).
        n_estimators (int): Number of trees in the Random Forest.

    Returns:
        model (RandomForestRegressor): Trained Random Forest model.
        y_test (pd.Series): True target values for the test set.
        y_pred (np.ndarray): Predicted target values for the test set.
        metrics (dict): A dictionary containing RMSE, MSE, MAE, and MAPE.
    """
    # Create lagged features
    for lag in range(1, n_lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)

    # Drop rows with NaN values resulting from lagging
    data = data.dropna()

    # Define predictors and target
    X = data[[f'lag_{i}' for i in range(1, n_lags + 1)]]
    y = data[target_col]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

    # Train the Random Forest model
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = (abs((y_test - y_pred) / y_test).mean()) * 100

    metrics = {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape
    }

    return model, y_test, y_pred, metrics


In [198]:
df_ph.columns

Index(['date', 'year', 'site', 'region', 'measure', 'data_value', 'unit',
       'statistic', 'dataset'],
      dtype='object')

In [199]:
df_ph.head()

,date,year,site,region,measure,data_value,unit,statistic,dataset
3,1998-01-20,1998,Taiaroa Head (50km offshore),Otago,pH,8.0948,NaN,Value,Munida
11,1998-02-16,1998,Taiaroa Head (50km offshore),Otago,pH,8.0906,NaN,Value,Munida
19,1998-03-17,1998,Taiaroa Head (50km offshore),Otago,pH,8.1026,NaN,Value,Munida
27,1998-04-15,1998,Taiaroa Head (50km offshore),Otago,pH,NaN,NaN,Value,Munida
35,1998-10-05,1998,Taiaroa Head (50km offshore),Otago,pH,8.0743,NaN,Value,Munida


In [200]:
df_ph['data_value']

3       8.0948
11      8.0906
19      8.1026
27         NaN
35      8.0743
         ...  
1051    8.0435
1059    8.0473
1067       NaN
1075    8.0493
1083       NaN
Name: data_value, Length: 136, dtype: float64

In [201]:
df_ph.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 3 to 1083
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        136 non-null    object 
 1   year        136 non-null    int64  
 2   site        136 non-null    object 
 3   region      136 non-null    object 
 4   measure     136 non-null    object 
 5   data_value  127 non-null    float64
 6   unit        0 non-null      object 
 7   statistic   136 non-null    object 
 8   dataset     136 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 10.6+ KB


In [202]:
df_ph = df_ph.drop(columns=['site', 'region', 'measure', 'unit', 'statistic', 'dataset'])
df_ph.head()

,date,year,data_value
3,1998-01-20,1998,8.0948
11,1998-02-16,1998,8.0906
19,1998-03-17,1998,8.1026
27,1998-04-15,1998,NaN
35,1998-10-05,1998,8.0743


In [ ]:
# Export df_ph to CSV
# df_ph.to_csv('ocean-acidification-ph-1998-2020.csv')

In [203]:
# fill in missing values using the custom average
df_ph_clean = impute_surrounding_average(df_ph, 'data_value')

/var/folders/h4/82kz_0j95dlf2rnsf3sbdtjh0000gn/T/ipykernel_26190/3015727988.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[idx] = (series[idx - 1] + series[idx + 1]) / 2
/var/folders/h4/82kz_0j95dlf2rnsf3sbdtjh0000gn/T/ipykernel_26190/3015727988.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[idx] = (series[idx - 1] + series[idx + 1]) / 2
/var/folders/h4/82kz_0j95dlf2rnsf3sbdtjh0000gn/T/ipykernel_26190/3015727988.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [204]:
df_ph_clean.isna().sum()

date          0
year          0
data_value    0
dtype: int64

In [205]:
df_ph_clean.shape

(136, 3)

In [206]:
# Call the modified function
model, y_test, y_pred, metrics = random_forest_time_series(
    data=df_ph_clean,
    target_col='data_value',
    n_lags=2,
    test_size=0.2,
    n_estimators=150
)

# Display metrics
print("Model Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")


Model Metrics:
MSE: 0.0004
RMSE: 0.0192
MAE: 0.0169
MAPE: 0.2102
